# Predicción de Especies de Pinguinos
## Clasificación multinomial
por Giovanni Rottoli

En este notebook vamos a ver como hacer clasificación, así como tambien hacer unos procesamientos adicionales con sklearn.


# Imports

In [ ]:
# Importamos la librería Pandas
import pandas as pd

Vamos a importar tambien el módulo preprocessing, la clase DecisionTreeClassifier, y las funciones plot_tree, accuracy_score, classification_report y train_test_split


In [ ]:
from sklearn import preprocessing # Para preprarar los datos
from sklearn.tree import DecisionTreeClassifier, plot_tree  # Modelado de los datos
from sklearn.metrics import accuracy_score, classification_report # Evaluación de los datos
from sklearn.model_selection import train_test_split # Separar entrenamiento de testing

# Data exploration

In [ ]:
data_path = 'https://raw.githubusercontent.com/yarrap/Penguin-Species-Prediction/refs/heads/main/penguins_size.csv'
print(f'Dataset path: {data_path}')

In [ ]:
df = pd.read_csv(data_path)
df.head()

Podemos ver que hay múltiples columnas, de las cuales buscamos predecir "species" en función del valor de los demás atributos.

In [ ]:
df.info()

Podemos ver que algunas tuplas tienen valores nulos. Procedemos a eliminar todas estas tuplas.

# Data preparation

In [ ]:
df_prep = df.copy()

## Limpieza

In [ ]:
df_prep = df_prep.dropna() # Eliminamos las filas con valores nulos

In [ ]:
df_prep.info()

Nos quedamos entonces con 334 tuplas limpias, sin valores perdidos.

Para utilizar el algoritmo de clasificación que provee scikit-learn, vamos a necesitar que las columnas sean numéricas. Para eso, vamos a codificar cada valor haciendo uso de la herramienta LabelEncoder

## Transformación

Con labelEncoder, construimos un modelo que nos va a permitir codificar los valores de entrada de forma numérica, y luego decodificarlos para obtener los valores originales.

Hay que hacer un modelo por cada atributo a codificar.

In [ ]:
# Instanciamos el modelo y lo entrenamos con fit.
le_species = preprocessing.LabelEncoder().fit(df_prep['species'])
# Aplicamos el modelo a la columna con transform, y asignamos los valores a la misma columna.
df_prep['species']=le_species.transform(df_prep['species'])

# Idem
le_sex = preprocessing.LabelEncoder().fit(df_prep['sex'])
df_prep['sex']=le_sex.transform(df_prep['sex'])

df_prep.head()

In [ ]:
target_names = list(le_species.classes_)
print(f"Clases para el atributo especie: {target_names }")
print(f"Clases para el atributo sexo: {list(le_sex.classes_) }")

Hay un valor que no esperábamos, el '.', es un valor anómalo que no podemos reemplazar, por lo tanto, lo eliminaremos:

In [ ]:
df_prep = df_prep[df_prep["sex"] != 0]
df_prep.info()

Ya vimos que una codificación numérica es útil solo cuando la variable es binomial o bien cuando existe un órden intrínseco en los valores. Ej: Joven, Adulto, Anciano. Podemos comparar valores por mayor o menor (relación de órden).

En caso de que no sea posible aplicarse, como en el caso de las íslas, una alternativa es usar One-hot encoding.

Para esto, pandas brinda una función: get_dummies, que automáticamente genera esta codificación. Luego deberemos eliminar la columna original y joinear la tabla original con las nuevas columnas generadas.

In [ ]:
for column in ['island']:
    # Obtengo columnas dummies por cada columna, en este caso solo 1, 'island'
    one_hot = pd.get_dummies(df_prep[column])
    # Elimino la columna original
    df_prep = df_prep.drop(column, axis = 1)
    # Anexto las columnas dummies a la tabla de datos.
    df_prep = df_prep.join(one_hot)

df_prep.head()

## Split

Vamos a separar los predictores del objetivo.

In [ ]:
target = 'species'
predictors = list(df_prep.columns)
predictors.remove(target)

In [ ]:
print(f'La columna objetivo es: {target}')
print(f'Los atributos de entrada son: \n {predictors}')

Vamos a entrenar el modelo con un conjunto de datos y probarlo con datos desconocidos. Para eso, vamos a partir el conjunto de datos en dos.

Nos quedamos con un 80% de los datos para entrenar el modelo y un 20% para las pruebas.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    df_prep[predictors],
    df_prep[target],
    test_size=0.2,)

In [ ]:
print(f'Tamaño del dataset de entrenamiento: {X_train.shape}' )
print(f'Tamaño del dataset de testing: {X_val.shape}' )

# Entrenamiento

Los entrenamientos con sklearn son todos parecidos.
1. Instanciamos un modelo
2. Entrenamos (fit)
3. Aplicamos (transform)

In [ ]:
params = {
    'criterion': 'gini',
    'max_depth': 2,
    'min_samples_leaf': 3
}

In [ ]:
clf = DecisionTreeClassifier(**params)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
_ = plot_tree(clf,
                   feature_names=predictors,
                   class_names=target_names,
                   filled=False,
                   precision=1)

In [ ]:
# Vemos qué valores nos devuelve el árbol para cada tupla de prueba.
y_pred = clf.predict(X_val)

In [ ]:
# Calculamos las métricas para ver qué tan bien funciona el método.
print(f'El valor de accuracy para el árbol de decisión generado es igual a {accuracy_score(y_val, y_pred)}')

In [ ]:
print(classification_report(y_val, y_pred, target_names=target_names))

# Actividades

1. Prueben de eliminar las islas del conjunto de datos de entrenamiento, y utilizar solo las caracteristicas fisiologicas de los pinguinos. ¿Cómo cambia el árbol?
2. Prueben cambiar la profundidad máxima y la cantidad minima de ejemplos por hoja. ¿Cómo cambia el árbol? ¿Cómo impacta en la medida de performance?